<a href="https://colab.research.google.com/github/Marcellinus08/Deep-Learning/blob/main/Costum_Models_and_Training_With_Tensoflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Library

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Data Loading

In [2]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

Custom Loss Function

In [3]:
class HuberLoss(keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)

    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss  = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

In [4]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1),
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
model.compile(loss=HuberLoss(2.0), optimizer="nadam", metrics=["mae"])
model.fit(X_train, y_train, epochs=5, validation_data=(X_valid, y_valid))

Epoch 1/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - loss: 1.5907 - mae: 1.5268 - val_loss: 0.6806 - val_mae: 0.8901
Epoch 2/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.6757 - mae: 0.9045 - val_loss: 0.6806 - val_mae: 0.8958
Epoch 3/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.6548 - mae: 0.8866 - val_loss: 0.6468 - val_mae: 0.8894
Epoch 4/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.6595 - mae: 0.8963 - val_loss: 0.6734 - val_mae: 0.9020
Epoch 5/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.6423 - mae: 0.8876 - val_loss: 0.6634 - val_mae: 0.8906


Custom Layers and Models

In [6]:
class MyDense(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)

    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
            name="kernel", shape=[batch_input_shape[-1], self.units],
            initializer="glorot_normal")
        self.bias = self.add_weight(
            name="bias", shape=[self.units], initializer="zeros")
        super().build(batch_input_shape)

    def call(self, X):
        return self.activation(X @ self.kernel + self.bias)

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "units": self.units,
                "activation": keras.activations.serialize(self.activation)}

In [7]:
class MyModel(keras.Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = MyDense(30, activation="relu")
        self.hidden2 = MyDense(30, activation="relu")
        self.output_layer = MyDense(1)

    def call(self, inputs):
        x = self.hidden1(inputs)
        x = self.hidden2(x)
        return self.output_layer(x)


In [8]:
custom_model = MyModel()
custom_model.compile(loss="mse", optimizer="adam")
print("\nMelatih model kustom...")
custom_model.fit(X_train, y_train, epochs=5, validation_data=(X_valid, y_valid))


Melatih model kustom...
Epoch 1/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 1.8431 - val_loss: 0.9516
Epoch 2/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4413 - val_loss: 1.7625
Epoch 3/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3757 - val_loss: 2.3373
Epoch 4/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3691 - val_loss: 1.8877
Epoch 5/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3604 - val_loss: 1.7058


Custom Training Loop

In [9]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1),
])

In [10]:
optimizer = keras.optimizers.Nadam(learning_rate=0.01)
loss_fn = keras.losses.MeanSquaredError()
mean_loss = keras.metrics.Mean()
metrics = [keras.metrics.MeanAbsoluteError()]

In [11]:
n_epochs = 5
batch_size = 32
n_steps = len(X_train) // batch_size

In [12]:
for epoch in range(1, n_epochs + 1):
    print(f"Epoch {epoch}/{n_epochs}")
    for step in range(1, n_steps + 1):
        # Ambil satu batch data
        indices = np.random.randint(len(X_train), size=batch_size)
        X_batch, y_batch = X_train[indices], y_train[indices]

        # Hitung gradien di dalam GradientTape
        with tf.GradientTape() as tape:
            y_pred = model(X_batch, training=True)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            loss = tf.add_n([main_loss] + model.losses) # Tambahkan loss dari regularisasi jika ada

        # Terapkan gradien
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        # Update metrik
        mean_loss(loss)
        for metric in metrics:
            metric(y_batch, y_pred)

        # Tampilkan status
        print(f"\rLangkah {step}/{n_steps} - loss: {mean_loss.result():.4f}", end="")

    # Reset metrik di akhir setiap epoch
    print() # Pindah baris
    for metric in [mean_loss] + metrics:
        metric.reset_state()

Epoch 1/5
Langkah 362/362 - loss: 1.1777
Epoch 2/5
Langkah 362/362 - loss: 0.4908
Epoch 3/5
Langkah 362/362 - loss: 0.4399
Epoch 4/5
Langkah 362/362 - loss: 0.4468
Epoch 5/5
Langkah 362/362 - loss: 0.8898


TensorFlow Functions


In [13]:
import timeit

# --- Versi Python Murni (Eager Execution) ---
# Perulangan ini berjalan di Python, yang membuatnya lambat.
def sum_up_to_eager(n):
    total = tf.constant(0, dtype=tf.int32)
    for i in range(n):
        total += i
    return total

# --- Versi TF Function ---
# @tf.function mengubah loop Python menjadi grafik komputasi.
@tf.function
def sum_up_to_graph(n):
    total = tf.constant(0, dtype=tf.int32)
    # AutoGraph akan mengubah loop ini menjadi tf.while_loop
    for i in tf.range(n): # Menggunakan tf.range agar di-trace
        total += i
    return total

# Jumlah iterasi untuk diuji
num_iterations = 10000

print(f"Menguji fungsi dengan perulangan sebanyak {num_iterations} kali...\n")

# Waktu eksekusi untuk fungsi Python murni
start_time_eager = timeit.default_timer()
sum_up_to_eager(num_iterations)
end_time_eager = timeit.default_timer()
print(f"Waktu Eager Execution (Python Loop): {end_time_eager - start_time_eager:.4f} detik")

# Waktu eksekusi untuk TF Function (termasuk tracing pertama kali)
start_time_graph = timeit.default_timer()
sum_up_to_graph(tf.constant(num_iterations))
end_time_graph = timeit.default_timer()
print(f"Waktu TF Function (Graph Loop): {end_time_graph - start_time_graph:.4f} detik")

Menguji fungsi dengan perulangan sebanyak 10000 kali...

Waktu Eager Execution (Python Loop): 1.1720 detik
Waktu TF Function (Graph Loop): 0.1761 detik
